In [3]:
!pip install -r /Users/aryanmathur/Desktop/41_m3_adobe/41_code_m3_adobe/task2/requirements(2)(1).txt

  Cloning https://github.com/huggingface/transformers.git to /private/var/folders/zv/2m5dbz0j7xlc8h11nms8zm100000gn/T/pip-req-build-ph89xf4w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/zv/2m5dbz0j7xlc8h11nms8zm100000gn/T/pip-req-build-ph89xf4w
  error: RPC failed; curl 56 LibreSSL SSL_read: LibreSSL/3.3.6: error:1404C3FC:SSL routines:ST_OK:sslv3 alert bad record mac, errno 0
  fatal: expected 'packfile'
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/folders/zv/2m5dbz0j7xlc8h11nms8zm100000gn/T/pip-req-build-ph89xf4w did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /private/var/fol

In [14]:
import torch
from transformers import Qwen2VLForConditionalGeneration, Qwen2VLProcessor
from qwen_vl_utils import process_vision_info
import os
import gc
import time
import json
import clip
from PIL import Image
import random

In [15]:
def clear_memory():
    # Delete variables if they exist in the current global scope
    if "inputs" in globals():
        del globals()["inputs"]
    if "model" in globals():
        del globals()["model"]
    if "processor" in globals():
        del globals()["processor"]
    if "trainer" in globals():
        del globals()["trainer"]
    if "peft_model" in globals():
        del globals()["peft_model"]
    if "bnb_config" in globals():
        del globals()["bnb_config"]
    time.sleep(2)

    # Garbage collection and clearing CUDA memory
    gc.collect()
    time.sleep(2)
    torch.cuda.empty_cache()
    torch.cuda.synchronize()
    time.sleep(2)
    gc.collect()
    time.sleep(2)

    print(f"GPU allocated memory: {torch.cuda.memory_allocated() / 1024**3:.2f} GB")
    print(f"GPU reserved memory: {torch.cuda.memory_reserved() / 1024**3:.2f} GB")

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [17]:
if torch.cuda.is_available():
    print(f"CUDA Device: {torch.cuda.get_device_name(0)}")
    print(f"Supports bfloat16: {torch.cuda.is_bf16_supported()}")
else:
    print("CUDA is not available on this system.")

CUDA is not available on this system.


In [18]:
# clear_memory() uncomment for cuda to clear gpu memory

In [19]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    device_map="auto",
    torch_dtype=torch.bfloat16,
)

processor = Qwen2VLProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Some parameters are on the meta device because they were offloaded to the cpu and disk.


In [20]:
def filter(img_path):

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model, preprocess = clip.load("ViT-B/32", device)

    geometric_and_structural_anomalies = [
        "Inconsistent object boundaries",
        "Discontinuous surfaces",
        "Non-manifold geometries in rigid structures",
        "Floating or disconnected components",
        "Asymmetric features in naturally symmetric objects",
        "Misaligned bilateral elements in animal faces",
        "Irregular proportions in mechanical components",
        "Impossible mechanical connections",
        "Inconsistent scale of mechanical parts",
        "Physically impossible structural elements",
        "Incorrect wheel geometry",
        "Implausible aerodynamic structures",
        "Misaligned body panels",
        "Impossible mechanical joints",
        "Anatomically impossible joint configurations",
        "Unnatural pose artifacts",
        "Biological asymmetry errors",
        "Excessive sharpness in certain image regions",
        "Unnaturally glossy surfaces"
    ]

    texture_and_surface_issues = [
        "Texture bleeding between adjacent regions",
        "Texture repetition patterns",
        "Over-smoothing of natural textures",
        "Artificial noise patterns in uniform surfaces",
        "Metallic surface artifacts",
        "Artificial enhancement artifacts",
        "Regular grid-like artifacts in textures",
        "Repeated element patterns",
        "Synthetic material appearance",
        "Artificial smoothness"
    ]

    lighting_and_reflection_problems = [
        "Unrealistic specular highlights",
        "Inconsistent material properties",
        "Multiple light source conflicts",
        "Missing ambient occlusion",
        "Incorrect reflection mapping",
        "Inconsistent shadow directions",
        "Glow or light bleed around object boundaries",
        "Incorrect Skin Tones",
        "Unnatural Lighting Gradients",
        "Dramatic lighting that defies natural physics",
        "Multiple inconsistent shadow sources"
    ]

    anatomical_and_biological_anomalies = [
        "Dental anomalies in mammals",
        "Anatomically incorrect paw structures",
        "Improper fur direction flows",
        "Unrealistic eye reflections",
        "Misshapen ears or appendages",
        "Anatomically impossible joint configurations",
        "Impossible foreshortening in animal bodies",
        "Exaggerated characteristic features"
    ]

    perspective_and_spatial_distortions = [
        "Incorrect perspective rendering",
        "Scale inconsistencies within single objects",
        "Spatial relationship errors",
        "Depth perception anomalies",
        "Fake depth of field",
        "Resolution inconsistencies within regions",
        "Artificial depth of field in object presentation",
        "Impossible mechanical joints"
    ]

    image_quality_issues = [
        "Over-sharpening artifacts",
        "Aliasing along high-contrast edges",
        "Blurred boundaries in fine details",
        "Jagged edges in curved structures",
        "Random noise patterns in detailed areas",
        "Loss of fine detail in complex structures",
        "Systematic color distribution anomalies",
        "Color coherence breaks",
        "Unnatural color transitions",
        "Frequency domain signatures"
    ]

    visual_artifacts_from_synthetic_image_generation = [
        "Ghosting effects: Semi-transparent duplicates of elements",
        "Cinematization Effects",
        "Movie-poster like composition of ordinary scenes",
        "Unnatural pose artifacts"
    ]

    occlusion_and_object_cutoff_issues = [
        "Abruptly cut off objects",
        "Inconsistent object boundaries"
    ]

    artifact_groups = {
        "Geometric and Structural Anomalies": geometric_and_structural_anomalies,
        "Texture and Surface Issues" : texture_and_surface_issues,
        "Lighting and Reflection Problems" : lighting_and_reflection_problems,
        "Anatomical and Biological Anomalies": anatomical_and_biological_anomalies,
        "Perspective and Spatial Distortions" : perspective_and_spatial_distortions,
        "Image Quality Issues" : image_quality_issues,
        "Visual Artifacts from Synthetic Image Generation": visual_artifacts_from_synthetic_image_generation,
        "Occlusion and Object Cut-off Issues": occlusion_and_object_cutoff_issues
    }


    img = Image.open(img_path)
    img_input = preprocess(img).unsqueeze(0).to(device)

    text_inputs = torch.cat([clip.tokenize(f"A photo of {artifact}") for artifact in artifact_groups]).to(device)

    with torch.no_grad():
        image_features = model.encode_image(img_input)
        text_features = model.encode_text(text_inputs)

    image_features /= image_features.norm(dim=-1, keepdim=True)
    text_features /= text_features.norm(dim=-1, keepdim=True)

    similarity = (image_features @ text_features.T).squeeze(0)

    artifact_group_names = list(artifact_groups.keys())

    top3_indices = similarity.topk(3).indices
    top3_scores = similarity[top3_indices].tolist()
    top3_categories = [artifact_group_names[idx] for idx in top3_indices]

    l = []
    for category in top3_categories:
        l.extend(artifact_groups[category])

    return l


In [21]:
def create_prompt(l):
    random.shuffle(l)

    prompt = f"""
Evaluate all artifact categories listed below and select MOST RELEVANT artifacts present in the image. Rank the detected artifacts in descending order of their intensity or confidence and provide an concise and image-specific explanation for each detected artifact in the following format:

<Artifact Category 1>: <Explanation for selection>
<Artifact Category 2>: <Explanation for selection>
...

Word Limit for the total explanation is 50 words

Unordered list of Artifact Categories (order does not imply priority):
{chr(10).join(f"{category}" for category in l)}

Your role is to evaluate **all categories equally**, without prioritizing based on their order in the list, and rank them based on their intensity or confidence. Provide accurate and reliable artifact detection results in the **Answer Format** described above.
    """

    return prompt


In [22]:
question="What artifacts can you detect in the provided image?"

In [23]:
def create_message(
    image_path,
    question=question,
):
    
    entry = [
                {
                    "role":"system",
                    "content": [
                        {
                            "type": "text", 
                            "text": create_prompt(filter(img_path= image_path)),
                        },
                    ],
                },
                {
                    "role":"user",
                    "content": [
                        {
                            "type": "image",
                            "image":  image_path,
                        },
                        {
                            "type": "text",
                            "text": question,
                        },
                    ],
                },
            ]
    return entry

In [24]:
   
def output(message): 

    text = processor.apply_chat_template(
        message, tokenize=False, add_generation_prompt=True
    )

    image_inputs, _ = process_vision_info(message)

    inputs = processor(
        text=[text],
        images=image_inputs,
        padding=True,
        return_tensors="pt",
    )
    inputs = inputs.to(device)


    generated_ids = model.generate(**inputs, max_new_tokens=150)
    generated_ids_trimmed = [
        out_ids[len(in_ids) :] for in_ids, out_ids in zip(inputs.input_ids, generated_ids)
    ]
    output_text = processor.batch_decode(
        generated_ids_trimmed, skip_special_tokens=True, clean_up_tokenization_spaces=False
    )
    
    return output_text

In [25]:
def fin_output(
    image_dir="/teamspace/studios/this_studio/fake", 
    output_file="./testing_fake.json"
):
    with open(output_file, "w") as f:
        results = []
        for i in sorted(os.listdir(image_dir)):
            image_path = os.path.join(image_dir, i)
            if i.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.gif')):
                file_name = i.split(".")[0]

                message = create_message(image_path=image_path)
                output_text = output(message=message)

                try:
                    lines = [line.strip() for line in output_text[0].split("\n") if line.strip()]
                    
                    output_dict = {}
                    for line in lines:
                        parts = line.split(":", 1)
                        if len(parts) == 2:
                            key = parts[0].strip()
                            value = parts[1].strip()
                            
                            if len(value.split()) > 80:
                                print(f"Skipping key '{key}' in {image_path} - word count exceeds 50")
                                continue
                            
                            output_dict[key] = value

                    json_data = {"explanation": output_dict}

                    final_data = {
                        "index": file_name,
                        **json_data
                    }
                    print(final_data)

                    results.append(final_data)

                except Exception as e:
                    print(f"Error processing {image_path}: {e}")
                    results.append({
                        "index": file_name,
                        "error": str(e)
                    })

        json.dump(results, f, indent=4)

In [26]:
fin_output()